# Definições base do tabuleiro
-> tabuleiro deve seguir as imagens base

-> tabuleiro deve possuir pontos de transição que permitam o agente ir de um andar ao outro

-> tabuleiro precisa gerar as comidinhas

In [1]:
def criar_tabuleiro():
    tabuleiro_base = []
    # Cria uma matriz (lista de listas) com o elemento especificado
    for i in range(11):
        tabuleiro_base.append([])
        for j in range(11):
            if i == 0 or j == 0 or i == 10 or j == 10:
                #tabuleiro_base[i].append("0")
                tabuleiro_base[i].append("■")
            else:
                
                tabuleiro_base[i].append('0')
                #tabuleiro_base[i].append('1')
    return tabuleiro_base

def imprimir_tabuleiro(tabuleiro):
    for linha in tabuleiro:
        print(' '.join(str(celula) for celula in linha))

# Criar o tabuleiro 11x11 com '1' representando as comidas
tabuleiro_pacman = criar_tabuleiro()

# Exibir o tabuleiro
imprimir_tabuleiro(tabuleiro_pacman)


■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■
■ 0 0 0 0 0 0 0 0 0 ■
■ 0 0 0 0 0 0 0 0 0 ■
■ 0 0 0 0 0 0 0 0 0 ■
■ 0 0 0 0 0 0 0 0 0 ■
■ 0 0 0 0 0 0 0 0 0 ■
■ 0 0 0 0 0 0 0 0 0 ■
■ 0 0 0 0 0 0 0 0 0 ■
■ 0 0 0 0 0 0 0 0 0 ■
■ 0 0 0 0 0 0 0 0 0 ■
■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■


In [2]:
def andar1():
    tabuleiro1 = criar_tabuleiro()
    paredes = [[2,1],[2,3],[2,5],[2,6],[2,7],[2,9],
               [3,3],
               [4,2], [4,3], [4,5],[4,7],[4,8],
               [5,5], [5,7],
               [6,4],[6,5], [6,7], [6,9],
               [7,7],
               [8,2], [8,3], [8,5],[8,6],[8,7], [8,9]]
    for parede in paredes:
        i, j = parede
        #tabuleiro1[i][j] = "0"
        tabuleiro1[i][j] = "■"
    #imprimir_tabuleiro(tabuleiro1)
    tabuleiro1[1][1] = "P"
    tabuleiro1[9][9] = "F"
    return tabuleiro1
andar1()

[['■', '■', '■', '■', '■', '■', '■', '■', '■', '■', '■'],
 ['■', 'P', '0', '0', '0', '0', '0', '0', '0', '0', '■'],
 ['■', '■', '0', '■', '0', '■', '■', '■', '0', '■', '■'],
 ['■', '0', '0', '■', '0', '0', '0', '0', '0', '0', '■'],
 ['■', '0', '■', '■', '0', '■', '0', '■', '■', '0', '■'],
 ['■', '0', '0', '0', '0', '■', '0', '■', '0', '0', '■'],
 ['■', '0', '0', '0', '■', '■', '0', '■', '0', '■', '■'],
 ['■', '0', '0', '0', '0', '0', '0', '■', '0', '0', '■'],
 ['■', '0', '■', '■', '0', '■', '■', '■', '0', '■', '■'],
 ['■', '0', '0', '0', '0', '0', '0', '0', '0', 'F', '■'],
 ['■', '■', '■', '■', '■', '■', '■', '■', '■', '■', '■']]

In [3]:
def andar2():
    tabuleiro2 = criar_tabuleiro()
    paredes = [[1,6],
    [2, 2], [2, 4], [2, 6], [2, 8],  # Terceira linha
    [3, 2], [3, 4], [3, 8], # Quarta linha
    [4,4], [4,5], [4,7],[4,8], # Quinta linha
    [5,2], [5,8], # Sexta linha
    [6, 2], [6,6],[6, 8],   # Sétima linha
    [7, 4], [7,6],  # Oitava linha
    [8,1],[8,2],[8,4],[8,6], [8,9],# Laterais da nona linha
    [9, 4]]  # Décima linha
    for parede in paredes:
        i, j = parede
        #tabuleiro2[i][j] = "0"
        tabuleiro2[i][j] = "■"
    #imprimir_tabuleiro(tabuleiro2)
    return tabuleiro2
andar2()

[['■', '■', '■', '■', '■', '■', '■', '■', '■', '■', '■'],
 ['■', '0', '0', '0', '0', '0', '■', '0', '0', '0', '■'],
 ['■', '0', '■', '0', '■', '0', '■', '0', '■', '0', '■'],
 ['■', '0', '■', '0', '■', '0', '0', '0', '■', '0', '■'],
 ['■', '0', '0', '0', '■', '■', '0', '■', '■', '0', '■'],
 ['■', '0', '■', '0', '0', '0', '0', '0', '■', '0', '■'],
 ['■', '0', '■', '0', '0', '0', '■', '0', '■', '0', '■'],
 ['■', '0', '0', '0', '■', '0', '■', '0', '0', '0', '■'],
 ['■', '■', '■', '0', '■', '0', '■', '0', '0', '■', '■'],
 ['■', '0', '0', '0', '■', '0', '0', '0', '0', '0', '■'],
 ['■', '■', '■', '■', '■', '■', '■', '■', '■', '■', '■']]

In [4]:
def andar3():
    tabuleiro3 = criar_tabuleiro()
    paredes = [[1,6],
    [2, 2], [2, 5], [2, 6],[2, 7], [2, 8],  # Terceira linha
    [3, 2], # Quarta linha
    [4,2], [4,4], [4,5],[4,6],[4,8], # Quinta linha
    [5,6], [5,8], # Sexta linha
    [6, 2], [6,4], [6,5], [6,6],   # Sétima linha
    [7, 2],  # Oitava linha
    [8,2],[8,3],[8,6],[8,7],[8,8],# Laterais da nona linha
    [9, 3]]  # Décima linha
    for parede in paredes:
        i, j = parede
        #tabuleiro3[i][j] = "0"
        tabuleiro3[i][j] = "■"
    #imprimir_tabuleiro(tabuleiro3)
    return tabuleiro3
    
andar3()

[['■', '■', '■', '■', '■', '■', '■', '■', '■', '■', '■'],
 ['■', '0', '0', '0', '0', '0', '■', '0', '0', '0', '■'],
 ['■', '0', '■', '0', '0', '■', '■', '■', '■', '0', '■'],
 ['■', '0', '■', '0', '0', '0', '0', '0', '0', '0', '■'],
 ['■', '0', '■', '0', '■', '■', '■', '0', '■', '0', '■'],
 ['■', '0', '0', '0', '0', '0', '■', '0', '■', '0', '■'],
 ['■', '0', '■', '0', '■', '■', '■', '0', '0', '0', '■'],
 ['■', '0', '■', '0', '0', '0', '0', '0', '0', '0', '■'],
 ['■', '0', '■', '■', '0', '0', '■', '■', '■', '0', '■'],
 ['■', '0', '0', '■', '0', '0', '0', '0', '0', '0', '■'],
 ['■', '■', '■', '■', '■', '■', '■', '■', '■', '■', '■']]

# Gerando agentes


In [5]:
from GameState import GameState
import numpy as np

In [6]:
gamestate = GameState([andar1(), andar2(), andar3()], True, [1,1,0], [9,9,0])

In [7]:
gamestate.board_state[2]

[['■', '■', '■', '■', '■', '■', '■', '■', '■', '■', '■'],
 ['■', '0', '0', '0', '0', '0', '■', '0', '0', '0', '■'],
 ['■', '0', '■', '0', '0', '■', '■', '■', '■', '0', '■'],
 ['■', '0', '■', '0', '0', '0', '0', '0', '0', '0', '■'],
 ['■', '0', '■', '0', '■', '■', '■', '0', '■', '0', '■'],
 ['■', '0', '0', '0', '0', '0', '■', '0', '■', '0', '■'],
 ['■', '0', '■', '0', '■', '■', '■', '0', '0', '0', '■'],
 ['■', '0', '■', '0', '0', '0', '0', '0', '0', '0', '■'],
 ['■', '0', '■', '■', '0', '0', '■', '■', '■', '0', '■'],
 ['■', '0', '0', '■', '0', '0', '0', '0', '0', '0', '■'],
 ['■', '■', '■', '■', '■', '■', '■', '■', '■', '■', '■']]

In [8]:
gamestate.is_terminal()

(True, 'Max')

In [ ]:
def MinMax(state:list, turn:bool, moves:list):
    best = []
    if turn:
        for i in gamestate.get_possible_moves():
            gamestate.score(state,gamestate.board_state, i)
    else:
            